In [ ]:
#Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt 

In [ ]:
#leer el dataframe 
dataframe=pd.read_csv('/content/97-fosfo-343.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'FOSFOI'], axis='columns')

In [ ]:
#defino las variables "X" y "y"
X = dataframe.drop(['FOSFOMICINA_'], axis='columns')
y = dataframe.iloc[:, -1]  
print(X)
print(y)

In [ ]:
#division de los datos en entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)
#print(X_train)
print(y_test)

In [ ]:
#convierto la variable "y" en log 
y_train1 = np.log2(y_train)
y_test1 = np.log2(y_test)
print(y_test1)
print(y_train1)

In [ ]:
#ajuste de hiperparametros por gridesearch
modelo1 = RandomForestRegressor()
n_estimators = [50, 100, 200, 300]
max_depth = [5, 6, 7, 8, 10]
min_samples_split = [2, 3, 5, 6] 
min_samples_leaf = [1, 2, 3] 
max_features = ['sqrt', 10, 20, 30, 45, 57]

hyperF = dict(max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, max_features = max_features)

gridF = GridSearchCV(modelo1, hyperF, cv = 10, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)
print('Best Params: ', gridF.best_params_)



In [ ]:
#definir el modelo
modelo = RandomForestRegressor(n_estimators = 100, max_depth= 6, min_samples_leaf= 2, min_samples_split= 2, max_features= 10)


In [ ]:
#tunning de hiperparametros (resultados con RMSE)
parametros = [10, 50, 100, 150, 200, 250] # parametros para n_estimators
train_scores, valid_scores = validation_curve(RandomForestRegressor(max_features= 'sqrt'), X = X_train, y=y_train1, param_name ="n_estimators", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
#parametros = [1, 2, 3, 4, 5, 6, 7, 10, 15, 20]  #parametros para max_depth
#train_scores, valid_scores = validation_curve(RandomForestRegressor(max_features= 'sqrt'), X = X_train, y=y_train1, param_name ="max_depth", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
#parametros = [1, 2, 3, 4, 5, 6, 7, 8, 10] #parametros para min_samples-split y min_samples_leaf
#train_scores, valid_scores = validation_curve(RandomForestRegressor(max_features= 'sqrt'), X = X_train, y=y_train1, param_name ="min_samples_split", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
train_scores_positive = np.array(train_scores) * -1
valid_scores_positive = np.array(valid_scores) * -1
train_mean = np.mean(train_scores_positive, axis=1)
train_std = np.std(train_scores_positive, axis=1)
val_mean = np.mean(valid_scores_positive, axis=1)
val_std = np.std(valid_scores_positive, axis=1)
print('Score train: ', train_scores_positive)
print('Score test: ', valid_scores_positive)
print('RMSE train: ', train_mean)
print('RMSE val: ', val_mean)
#grafica de validacion RMSE
plt.plot(parametros, train_mean, color='blue', marker='o', markersize=5, label='training score')
plt.fill_between(parametros, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(parametros, val_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation score')
plt.fill_between(parametros, val_mean + val_std, val_mean - val_std, alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
#plt.xscale('linear')
#plt.xticks(parametros)
plt.title('FOSFOMICINA 100%')
plt.legend(loc='upper righ')
plt.xlabel('Número de EStimadores')
plt.ylabel('RMSE')
plt.ylim([-1, 5])
plt.show()

In [ ]:
#entreno modelo y realiazo predicciones 
modelo.fit(X_train, y_train1) #entreno
y_pred = modelo.predict(X_test)
#hago predicciones 
print(y_pred)
y_pred1 = 2**np.round(modelo.predict(X_test))
print(y_pred1)
verdaderos = y_test.to_numpy()
print(verdaderos)

In [ ]:
#obtengo los valores que muestran el desempeño del modelo
print('Mean Absolute Error_test:', metrics.mean_absolute_error(y_test1, y_pred))
print('Mean Absolute Error_train:', metrics.mean_absolute_error(y_train1, modelo.predict(X_train)))
print('Mean Squared Error_test:', metrics.mean_squared_error(y_test1, y_pred))
print('Mean Squared Error_train:', metrics.mean_squared_error(y_train1, modelo.predict(X_train)))
print('Root Mean Squared Error_test:', np.sqrt(metrics.mean_squared_error(y_test1, y_pred)))
print('RMSE_train:', np.sqrt(metrics.mean_squared_error(y_train1, modelo.predict(X_train))))
print('r2_train:', metrics.r2_score(y_train1, modelo.predict(X_train)))
print('r2_teste:', metrics.r2_score(y_test1, (y_pred)))

In [ ]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y_test, y_pred1, dill=2):
    frac = np.array(y_test) / np.array(y_pred1)
    return ((frac >= 1/dill) & (frac <= dill)).mean()

In [ ]:
#saco accuracy con +/- 1 dilucion, VME y ME
acc2 = within_dillution(y_test, y_pred1, dill=2)
acc4 = within_dillution(y_test, y_pred1, dill=4)
threshold = 2
threshold1 = 2 #colacel el punto de corte para el antibiotico que esta probando
predDF = pd.DataFrame({'y_pred1':y_pred1, 'y_test':y_test})
vme = len(predDF[(predDF.y_pred1<=threshold) & (predDF.y_test>threshold1)])
me = len(predDF[(predDF.y_pred1>threshold1) & (predDF.y_test<=threshold)])
print(vme)
print(me)
print(acc2)
print(acc4)